# TNT Multi-Species: LAOS

**Objectives:**
- Fit TNT multi-species model to large-amplitude oscillatory shear data
- Understand species-resolved nonlinear dynamics
- Analyze different Weissenberg numbers per species
- Visualize Lissajous curves and harmonic content
- Compare NLSQ and Bayesian inference

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt
import arviz as az

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTMultiSpecies

sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_pnas_laos,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_multi_species_param_names,
    plot_multi_species_spectrum,
    plot_mode_decomposition,
)

## Theory: Multi-Species LAOS Dynamics

In large-amplitude oscillatory shear, each bond species experiences different levels of nonlinearity:

**Strain input:**
$$\gamma(t) = \gamma_0 \sin(\omega t)$$

**Per-species Weissenberg number:**
$$Wi_i = \gamma_0 \omega \tau_{b,i}$$

**Key physics:**
- Each species has its own nonlinear onset
- Fast species (short $\tau_{b,0}$): May remain in linear regime ($Wi_0 < 1$)
- Slow species (long $\tau_{b,1}$): May enter nonlinear regime ($Wi_1 > 1$)
- Total stress is superposition of linear and nonlinear contributions

**Nonlinearity regimes:**
- $Wi_i \ll 1$: Linear response (fundamental harmonic only)
- $Wi_i \sim 1$: Weak nonlinearity (3rd harmonic emerges)
- $Wi_i \gg 1$: Strong nonlinearity (higher harmonics, large distortions)

**Lissajous curves:**
- Elastic Lissajous: $\sigma$ vs $\gamma$ (ellipse → distorted loop)
- Viscous Lissajous: $\sigma$ vs $\dot{\gamma}$ (ellipse → distorted loop)
- Distortion quantifies nonlinearity

**Harmonic analysis:**
- Linear: Only $\omega$ (fundamental)
- Nonlinear: $3\omega, 5\omega, 7\omega, ...$ (odd harmonics)
- Harmonic intensities measure nonlinearity strength

## Load Data

In [ ]:
time_data, strain, stress = load_pnas_laos(omega=1.0, strain_amplitude_index=5)
gamma_0 = float(np.max(np.abs(strain)))

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.4f} to {time_data.max():.2f} s")
print(f"Strain amplitude: γ₀ = {gamma_0:.4f}")
print(f"Frequency: ω = 1.0 rad/s")
print(f"Stress range: {stress.min():.2f} to {stress.max():.2f} Pa")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(time_data, strain, label='Strain', linewidth=2)
ax1.plot(time_data, stress, label='Stress', linewidth=2)
ax1.set_xlabel('Time [s]', fontsize=12)
ax1.set_ylabel('Strain / Stress [Pa]', fontsize=12)
ax1.set_title('LAOS Time Series', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(strain, stress, 'o-', markersize=3, linewidth=1)
ax2.set_xlabel('Strain', fontsize=12)
ax2.set_ylabel('Stress [Pa]', fontsize=12)
ax2.set_title('Elastic Lissajous Curve', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
# CI mode: Skip slow NLSQ fit - use reasonable defaults
FAST_MODE = os.environ.get("FAST_MODE", "0") == "1"

model = TNTMultiSpecies(n_species=2)
param_names = get_tnt_multi_species_param_names(n_species=2)
print(f"Parameters: {param_names}")

# Oscillation frequency (matches data loading)
omega = 1.0  # rad/s

if FAST_MODE:
    print("FAST_MODE: Using default parameters (NLSQ fit for LAOS is slow)")
    # Set reasonable parameters for LAOS
    model.parameters.set_value('G_0', 500.0)  # Pa
    model.parameters.set_value('tau_b_0', 1.0)  # s
    model.parameters.set_value('G_1', 100.0)  # Pa
    model.parameters.set_value('tau_b_1', 0.1)  # s
    model.parameters.set_value('eta_s', 0.01)  # Pa·s
    nlsq_time = 0.0
else:
    start_time = time.time()
    model.fit(time_data, stress, test_mode="laos", gamma_0=gamma_0, omega=omega, method='scipy')
    nlsq_time = time.time() - start_time
    print(f"\nNLSQ converged: (check via model state)")
    print(f"Optimization time: {nlsq_time:.2f} s")

print(f"\nFitted parameters:")
for name in param_names:
    print(f"  {name}: {model.parameters.get_value(name):.6e}")

In [ ]:
time_pred = np.linspace(time_data.min(), time_data.max(), 1000)
stress_pred = model.predict(time_pred, gamma_0=gamma_0, omega=1.0, test_mode="laos")
strain_pred = gamma_0 * np.sin(1.0 * time_pred)
stress_fit = model.predict(time_data, gamma_0=gamma_0, omega=1.0, test_mode="laos")

fit_metrics = compute_fit_quality(stress, stress_fit)
print(f"\nFit quality:")
print(f"  R² = {fit_metrics['R2']:.6f}")
print(f"  RMSE = {fit_metrics['RMSE']:.6e}")
print(f"  NRMSE = {fit_metrics['NRMSE']:.6f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(time_data, stress, 'o', label='Data', markersize=4, alpha=0.7)
ax1.plot(time_pred, stress_pred, '-', label='NLSQ Fit', linewidth=2)
ax1.set_xlabel('Time [s]', fontsize=12)
ax1.set_ylabel('Stress [Pa]', fontsize=12)
ax1.set_title(f'TNT Multi-Species LAOS (R² = {fit_metrics['R2']:.4f})', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(strain, stress, 'o', label='Data', markersize=4, alpha=0.7)
ax2.plot(strain_pred, stress_pred, '-', label='NLSQ Fit', linewidth=2)
ax2.set_xlabel('Strain', fontsize=12)
ax2.set_ylabel('Stress [Pa]', fontsize=12)
ax2.set_title('Lissajous Curve', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")
plt.close('all')

## Physical Analysis: Species-Resolved Nonlinearity

In [ ]:
G_0 = model.parameters.get_value('G_0')
tau_b_0 = model.parameters.get_value('tau_b_0')
G_1 = model.parameters.get_value('G_1')
tau_b_1 = model.parameters.get_value('tau_b_1')
eta_s = model.parameters.get_value('eta_s')

omega = 1.0
Wi_0 = gamma_0 * omega * tau_b_0
Wi_1 = gamma_0 * omega * tau_b_1

print("\nSpecies-resolved Weissenberg numbers:")
print(f"\nGlobal parameters:")
print(f"  Strain amplitude: γ₀ = {gamma_0:.4f}")
print(f"  Frequency: ω = {omega:.2f} rad/s")

print(f"\nSpecies 0 (fast):")
print(f"  G_0 = {G_0:.3e} Pa")
print(f"  tau_b_0 = {tau_b_0:.3e} s")
print(f"  Wi_0 = γ₀ · ω · tau_b_0 = {Wi_0:.4f}")
if Wi_0 < 0.5:
    print(f"  → LINEAR regime (Wi_0 << 1)")
elif Wi_0 < 2:
    print(f"  → WEAKLY NONLINEAR regime (Wi_0 ~ 1)")
else:
    print(f"  → STRONGLY NONLINEAR regime (Wi_0 >> 1)")

print(f"\nSpecies 1 (slow):")
print(f"  G_1 = {G_1:.3e} Pa")
print(f"  tau_b_1 = {tau_b_1:.3e} s")
print(f"  Wi_1 = γ₀ · ω · tau_b_1 = {Wi_1:.4f}")
if Wi_1 < 0.5:
    print(f"  → LINEAR regime (Wi_1 << 1)")
elif Wi_1 < 2:
    print(f"  → WEAKLY NONLINEAR regime (Wi_1 ~ 1)")
else:
    print(f"  → STRONGLY NONLINEAR regime (Wi_1 >> 1)")

print(f"\nNonlinearity interpretation:")
if Wi_0 < 0.5 and Wi_1 > 2:
    print(f"  Fast species: Linear (contributes fundamental harmonic)")
    print(f"  Slow species: Nonlinear (contributes higher harmonics)")
    print(f"  → Mixed linear/nonlinear response")
elif Wi_0 > 2 and Wi_1 > 2:
    print(f"  Both species: Strongly nonlinear")
    print(f"  → Full nonlinear response with rich harmonic content")
else:
    print(f"  Mixed regime: Complex interplay of linear and nonlinear contributions")

## Harmonic Analysis via FFT

In [ ]:
# FFT of stress signal to extract harmonics
N = len(stress_pred)
dt = time_pred[1] - time_pred[0]
fft_stress = np.fft.fft(stress_pred)
freqs = np.fft.fftfreq(N, dt) * 2 * np.pi  # Convert to rad/s

# Positive frequencies only
positive_freqs = freqs[:N//2]
fft_magnitude = np.abs(fft_stress[:N//2]) / N * 2

# Find fundamental and harmonics (ω, 3ω, 5ω, ...)
omega_fundamental = 1.0
harmonics = [1, 3, 5, 7, 9]
harmonic_amplitudes = []

for n in harmonics:
    target_freq = n * omega_fundamental
    idx = np.argmin(np.abs(positive_freqs - target_freq))
    harmonic_amplitudes.append(fft_magnitude[idx])

print("\nHarmonic content (FFT analysis):")
for n, amp in zip(harmonics, harmonic_amplitudes):
    print(f"  {n}ω: {amp:.3e} Pa (amplitude)")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# FFT spectrum
ax1.semilogy(positive_freqs, fft_magnitude, linewidth=1)
for n in harmonics:
    ax1.axvline(n * omega_fundamental, color='r', linestyle='--', alpha=0.5, linewidth=1)
ax1.set_xlabel('Frequency [rad/s]', fontsize=12)
ax1.set_ylabel('Amplitude [Pa]', fontsize=12)
ax1.set_title('FFT Spectrum', fontsize=14)
ax1.set_xlim([0, 10])
ax1.grid(True, alpha=0.3)

# Harmonic bar chart
ax2.bar([str(n) + 'ω' for n in harmonics], harmonic_amplitudes)
ax2.set_xlabel('Harmonic', fontsize=12)
ax2.set_ylabel('Amplitude [Pa]', fontsize=12)
ax2.set_title('Harmonic Amplitudes', fontsize=14)
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")
plt.close('all')

# Nonlinearity quantification
I3_1 = harmonic_amplitudes[1] / harmonic_amplitudes[0]  # 3ω / 1ω
print(f"\nNonlinearity metric:")
print(f"  I₃/₁ = (3ω amplitude) / (1ω amplitude) = {I3_1:.4f}")
if I3_1 < 0.01:
    print(f"  → Predominantly linear response")
elif I3_1 < 0.1:
    print(f"  → Weakly nonlinear response")
else:
    print(f"  → Strongly nonlinear response")

## Species Contributions Discussion

In [ ]:
fig = plot_multi_species_spectrum(model)
plt.close("all")
plt.close('all')

print("\nSpecies contributions to LAOS response:")
print(f"\nFast species (Wi_0 = {Wi_0:.2f}):")
if Wi_0 < 0.5:
    print(f"  - Contributes mainly to fundamental harmonic (linear)")
    print(f"  - Minimal higher harmonic content")
    print(f"  - Acts as elastic background")
else:
    print(f"  - Contributes to both fundamental and higher harmonics")
    print(f"  - Exhibits nonlinear stress response")

print(f"\nSlow species (Wi_1 = {Wi_1:.2f}):")
if Wi_1 > 2:
    print(f"  - Strongly nonlinear")
    print(f"  - Major contributor to 3ω, 5ω, ... harmonics")
    print(f"  - Distorts Lissajous curve")
else:
    print(f"  - Contributes to fundamental harmonic")
    print(f"  - Limited higher harmonic content")

print(f"\nTotal response:")
print(f"  Superposition of species contributions")
print(f"  Nonlinearity emerges from species with Wi > 1")
print(f"  Different species may be in different regimes simultaneously")

## Bayesian Inference

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    result_bayes = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    result_bayes = model.fit_bayesian(
        time_data, stress,
        test_mode='laos',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(result_bayes, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


## ArviZ Diagnostics: Trace Plots

In [ ]:
# Skip trace plot in CI mode
if not FAST_MODE:
    idata = az.from_dict(posterior={name: result_bayes.posterior_samples[name][None, :] for name in param_names})
    axes = az.plot_trace(idata, var_names=param_names, compact=True)
    plt.tight_layout()
    plt.close("all")
    plt.close('all')
else:
    print("FAST_MODE: Skipping trace plot")


## ArviZ Diagnostics: Posterior Distributions

In [ ]:
# Skip ArviZ plot in CI mode
if not FAST_MODE:
    fig = az.plot_posterior(idata, var_names=param_names, hdi_prob=0.95)
    plt.tight_layout()
    plt.close("all")
    plt.close('all')
else:
    print("FAST_MODE: Skipping ArviZ plot")


## ArviZ Diagnostics: Pair Plot

In [ ]:
# Skip pair plot in CI mode
if not FAST_MODE:
    axes = az.plot_pair(idata, var_names=param_names, divergences=True)
    plt.close("all")
    plt.close('all')
else:
    print("FAST_MODE: Skipping pair plot")


## NLSQ vs Bayesian Parameter Comparison

In [ ]:
print_parameter_comparison(model, result_bayes.posterior_samples, param_names)

## Posterior Predictive: LAOS Response

In [ ]:
# CI mode: Skip slow posterior predictive (200 ODE solves)
if FAST_MODE:
    print("FAST_MODE: Skipping posterior predictive (200 predictions would take >300s)")
    # Use NLSQ fit for visualization instead
    y_pred_mean = model.predict(time_pred, gamma_0=gamma_0, omega=1.0, test_mode="laos")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1.plot(time_data, stress, 'o', label='Data', markersize=4, alpha=0.7, zorder=3)
    ax1.plot(time_pred, y_pred_mean, '-', label='NLSQ Fit', linewidth=2, zorder=2)
    ax1.set_xlabel('Time [s]', fontsize=12)
    ax1.set_ylabel('Stress [Pa]', fontsize=12)
    ax1.set_title('LAOS Time Series (CI Mode)', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(strain, stress, 'o', label='Data', markersize=4, alpha=0.7, zorder=3)
    ax2.plot(strain_pred, y_pred_mean, '-', label='NLSQ Fit', linewidth=2, zorder=2)
    ax2.set_xlabel('Strain', fontsize=12)
    ax2.set_ylabel('Stress [Pa]', fontsize=12)
    ax2.set_title('Lissajous Curve (CI Mode)', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.close("all")
    plt.close('all')
else:
    posterior = result_bayes.posterior_samples
    n_draws = min(200, NUM_SAMPLES)
    draw_indices = np.linspace(0, NUM_SAMPLES - 1, n_draws, dtype=int)

    x_pred = time_pred
    y_pred_samples = []

    for i in draw_indices:
        # Set parameters from posterior draw
        for name in param_names:
            model.parameters.set_value(name, posterior[name][i])
        # Predict with current parameters
        y_pred_i = model.predict(x_pred, gamma_0=gamma_0, omega=1.0, test_mode="laos")
        y_pred_samples.append(np.array(y_pred_i))

    y_pred_samples = np.array(y_pred_samples)
    y_pred_mean = np.mean(y_pred_samples, axis=0)
    y_pred_lower = np.percentile(y_pred_samples, 2.5, axis=0)
    y_pred_upper = np.percentile(y_pred_samples, 97.5, axis=0)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    ax1.plot(time_data, stress, 'o', label='Data', markersize=4, alpha=0.7, zorder=3)
    ax1.plot(time_pred, y_pred_mean, '-', label='Posterior Mean', linewidth=2, zorder=2)
    ax1.fill_between(time_pred, y_pred_lower, y_pred_upper, alpha=0.3, label='95% CI', zorder=1)
    ax1.set_xlabel('Time [s]', fontsize=12)
    ax1.set_ylabel('Stress [Pa]', fontsize=12)
    ax1.set_title('Posterior Predictive: LAOS Time Series', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    ax2.plot(strain, stress, 'o', label='Data', markersize=4, alpha=0.7, zorder=3)
    ax2.plot(strain_pred, y_pred_mean, '-', label='Posterior Mean', linewidth=2, zorder=2)
    ax2.fill_between(strain_pred, y_pred_lower, y_pred_upper, alpha=0.3, label='95% CI', zorder=1)
    ax2.set_xlabel('Strain', fontsize=12)
    ax2.set_ylabel('Stress [Pa]', fontsize=12)
    ax2.set_title('Posterior Predictive: Lissajous Curve', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.close("all")
    plt.close('all')

## Physical Interpretation

**Species-resolved nonlinearity:**
- Each species has independent Weissenberg number $Wi_i = \gamma_0 \omega \tau_{b,i}$
- Fast species (short $\tau_{b,0}$) may be linear while slow species is nonlinear
- Or both may be nonlinear with different degrees of distortion
- Total stress is superposition of linear and nonlinear contributions

**Harmonic content:**
- Linear species: Fundamental harmonic only (1ω)
- Nonlinear species: Higher odd harmonics (3ω, 5ω, 7ω, ...)
- FFT analysis reveals nonlinearity strength
- $I_{3/1}$ ratio quantifies third harmonic contribution

**Lissajous distortion:**
- Linear: Ellipse
- Nonlinear: Distorted loop (S-shape, asymmetry)
- Distortion magnitude correlates with Wi

**Uncertainty quantification:**
- Bayesian posteriors capture parameter correlations
- LAOS data constrains nonlinear parameters
- Complementary to SAOS (linear regime) data

## Save Results

In [ ]:
save_tnt_results(model, result_bayes, "multi_species", "laos", param_names)
print("Results saved successfully.")

## Key Takeaways

1. **Species-resolved Wi**: Each species has independent nonlinearity onset
2. **Mixed regimes**: Fast species may be linear while slow species is nonlinear
3. **Harmonic analysis**: FFT reveals contribution of higher harmonics (nonlinearity signature)
4. **Lissajous distortion**: Visualizes nonlinearity in stress-strain space
5. **Superposition**: Total LAOS response is sum of species contributions
6. **Complementary data**: LAOS constrains nonlinear parameters, complements SAOS
7. **Bayesian inference**: Quantifies uncertainty in multi-species nonlinear dynamics